# Generic Magnetic System Explorer

Interactive notebook for exploring magnetic systems with real-time visualization.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spinlab.analysis.visualization import SpinVisualizer
import time

# Configure matplotlib for notebook
%matplotlib inline

# Initialize visualizer
viz = SpinVisualizer()
print("Generic Magnetic System Explorer Ready!")

## 1. Quick System Setup

Create any magnetic system with minimal code:

In [ ]:
# Quick system builder
build_system = viz.create_system_builder()

# Create Fe system (change material here: 'Fe', 'Ni', 'Co', 'Mn')
material = 'Fe'
system = build_system(
    material=material, 
    size=(16, 16, 1),  # 16x16 2D system
    J=-0.02,          # Exchange coupling
    B_field=[0, 0, 0.1]  # Small z-field
)

print(f"{material} system created: {len(system.positions)} sites")
print(f"Initial energy: {system.total_energy():.4f} eV")

## 2. Static Spin Configuration

View current spin state:

In [ ]:
# Plot current configuration
viz.plot_spin_grid_skyrmion_style(
    system.spin_config,
    title=f"{material} Spin Configuration (Random Initial State)",
    figsize=(8, 8)
)

## 3. Simulation with Real-Time Monitoring

Run simulation with real-time updates of energy, magnetization, and spin configuration:

In [ ]:
# Create monitor
monitor = viz.create_live_monitor(system, figsize=(15, 5))

# Run simulation
result = viz.run_live_simulation(
    system, 
    monitor, 
    temperature=100,    # Temperature in K
    n_steps=1000,       # Total MC steps
    update_every=50     # Update frequency
)

print(f"\nFinal state: E = {result['final_energy']:.4f} eV, |M| = {np.linalg.norm(result['final_magnetization']):.3f}")

## 4. Temperature Sweep with Phase Diagram

Explore phase transitions with real-time plotting:

In [ ]:
# Temperature range for phase diagram
T_range = np.linspace(50, 300, 15)  # 50K to 300K

# Temperature sweep
phase_results = viz.temperature_sweep_live(
    system, 
    T_range, 
    steps_per_T=200  # MC steps per temperature
)

print(f"\nPhase diagram complete for {material}!")

## 5. Final Equilibrated Configuration

View the final spin state after equilibration:

In [ ]:
# Plot final equilibrated state
viz.plot_spin_grid_skyrmion_style(
    system.spin_config,
    title=f"{material} Final Equilibrated State",
    figsize=(8, 8)
)

# Summary
print(f"\n{material} System Summary:")
print(f"   System size: {len(system.positions)} sites")
print(f"   Final energy: {system.total_energy():.4f} eV")
print(f"   Final |M|: {np.linalg.norm(system.total_magnetization()):.3f}")

## 6. Try Different Materials

Quick comparison of different magnetic materials:

In [ ]:
# Compare different materials
materials = ['Fe', 'Ni', 'Co']
fig, axes = plt.subplots(1, len(materials), figsize=(15, 5))

for i, mat in enumerate(materials):
    # Create system
    sys = build_system(material=mat, size=(12, 12, 1), J=-0.02)
    
    # Quick equilibration
    from spinlab.monte_carlo import MonteCarlo
    mc = MonteCarlo(sys, temperature=150)
    mc.run(n_steps=500, verbose=False)
    
    # Plot on subplot
    plt.sca(axes[i])
    
    # Extract spin data for plotting
    spins = sys.spin_config
    n_spins = len(spins)
    grid_size = int(np.sqrt(n_spins))
    
    # Create grid positions
    x_1d = np.arange(grid_size)
    y_1d = np.arange(grid_size)
    X, Y = np.meshgrid(x_1d, y_1d)
    x, y = X.flatten(), Y.flatten()
    
    # Spin components
    sx, sy, sz = spins[:, 0], spins[:, 1], spins[:, 2]
    r = np.sqrt(sx**2 + sy**2 + sz**2)
    background_colors = sz / r
    
    # Simple plotting (no tricontourf for subplot)
    plt.scatter(x, y, c=background_colors, cmap='jet', s=15, vmin=-1, vmax=1)
    plt.quiver(x, y, sx, sy, color='black', scale=30, width=0.002)
    plt.title(f'{mat} (T=150K)')
    plt.axis('equal')
    plt.xlim(-1, grid_size)
    plt.ylim(-1, grid_size)

plt.tight_layout()
plt.show()
print("Material comparison complete!")

## 7. Performance Information

Check simulation speed and system capabilities:

In [ ]:
# Quick speed test
print("Performance Test:")
test_system = build_system('Fe', size=(20, 20, 1))
from spinlab.monte_carlo import MonteCarlo

mc = MonteCarlo(test_system, temperature=100)
start_time = time.time()
mc.run(n_steps=100, verbose=False)
elapsed = time.time() - start_time

speed = 100 / elapsed
print(f"Speed: {speed:.1f} sweeps/s for {len(test_system.positions)} sites")
print(f"Estimated time for 10k sweeps: {10000/speed:.1f} seconds")

# Numba status
from spinlab.core.fast_ops import HAS_NUMBA
print(f"Numba acceleration: {'Active' if HAS_NUMBA else 'Not available'}")